# How to handle connections to devices

## Automatically

By default, connections are handled automatically:

In [1]:
from nornir.core import InitNornir
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import napalm_get

In [2]:
nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=napalm_get,
    getters=["facts"]
)
print_result(r)

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Manually

In some circumstances, you may want to manage connections manually. To do so you can use 
[open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection), [close_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connection), [close_connections](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connections) and [Nornir.close_connections](../ref/api/nornir.rst#nornir.core.Nornir.close_connections). For instance:

In [3]:
def task_manages_connection_manually(task):
    task.host.open_connection("napalm")
    r = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    task.host.close_connection("napalm")
    
nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_connection_manually,
)
print_result(r)

task_manages_connection_manually************************************************
* rtr00 ** changed : False *****************************************************
vvvv task_manages_connection_manually ** changed : False vvvvvvvvvvvvvvvvvvvvvvv ERROR
Traceback (most recent call last):
  File "/Users/dbarroso/workspace/nornir/nornir/core/task.py", line 63, in start
    r = self.task(self, **self.params)
  File "<ipython-input-3-27058784ffcd>", line 5, in task_manages_connection_manually
    getters=["facts"]
  File "/Users/dbarroso/workspace/nornir/nornir/core/task.py", line 100, in run
    raise Exception(msg)
Exception: ('You have to call this after setting host and nornir attributes. ', 'You probably called this from outside a nested task')

^^^^ END task_manages_connection_manually ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Specifying connection parameters

When using the [open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection) you can specify any parameters you want. If you don't, or if you let nornir open the connection automatically, nornir will read those parameters from the inventory. You can specify standard attributes at the object level if you want to reuse them across different connections or you can override them for each connection:

```
my_host:
    # standard parameters that will be reused across different connections
    username: my_user
    password: my_password
    port: 22
    platform: linux
    napalm_options:
        # standard parameters that will be used only for the napalm connection
        # missing parameters will be read from the top level
        port: 443
        platform: eos
        connection_options:
            # advanced options that are specific to this connection type
            optional_args:
                eos_autoComplete: true
    netmiko_options:
        platform: arista_eos
        connection_options:
            global_delay: 2
```